In [1]:
import pandas as pd
import numpy as np

In [2]:
train_dataset = np.load('./data/train_image_dust.npy')
test_dataset = np.load('./data/test_image_dust.npy')

print(train_dataset.shape)
print(test_dataset.shape)

(2160, 10, 20)
(720, 10, 20)


# Sampler

In [3]:
def sampler_stamp2(data, stamp, lag, step, temp=False):

    num_row = len(data)
    data_x, data_y = [], []
    for idx in range(num_row-stamp*lag-step):
        y = np.array(data[stamp*lag+idx+step-1])
        if not temp:
            x = np.transpose(data[[stamp*i+idx for i in range(lag)],:], [1,2,0])
            data_x.append(x)
#             y = np.transpose(y, [1,2,0])
        data_y.append(y)
    if not temp:
        data_y = np.expand_dims(data_y, axis=-1)
    else:
        data_y = np.expand_dims(data_y, axis=1)
#         data_y = np.reshape(data_y, (np.shape(data_y)[0], np.shape(data_y)[1], np.shape(data_y)[2], 1))
    print("Sampler Return", np.shape(data_x), np.shape(data_y))
    
    if not temp:
        return np.array(data_x), np.array(data_y)
    else:
        return np.array(data_y)

# Make Temporal Information

In [4]:
print("Train shape: ", np.shape(train_dataset), ", Test shape: ", np.shape(test_dataset))

# Setting Some Parameters 
num_train, num_test = np.shape(train_dataset)[0], np.shape(test_dataset)[0]
num_row = num_train + num_test
print('num_row: ', num_row)

Train shape:  (2160, 10, 20) , Test shape:  (720, 10, 20)
num_row:  2880


In [5]:
### Initialize numpy array of temporal information (one-hot encoding)
datasets_1h = np.zeros([num_row, 24])
datasets_dow = np.zeros([num_row, 7])

# 더미화 하기
# 1hour, and day-of-week index are calculated below
for i in range(num_row):
    idx_1h = int(int(i)%24)
    idx_dow = int(int(i/24)%7)
    datasets_1h[i,idx_1h] = 1
    datasets_dow[i, idx_dow] = 1

In [6]:
def train_test_split(data, idx):
    return data[:idx], data[idx:]

In [7]:
# Split Train & Test Period

train_index = num_train #144 # 120 days and 144 time index
hour_1_train, hour_1_test = train_test_split(datasets_1h, train_index)
dow_train, dow_test = train_test_split(datasets_dow, train_index)

print(hour_1_train.shape, hour_1_test.shape)
print(dow_train.shape, dow_test.shape)

(2160, 24) (720, 24)
(2160, 7) (720, 7)


# Save Final Data

In [8]:
def save_data_stamp2(STAMP, LAG, STEP):
    # train 
    x_train, y_train = sampler_stamp2(train_dataset, stamp=STAMP, lag=LAG, step=STEP, temp=False)    
    hour_1_train_y = sampler_stamp2(hour_1_train, stamp=STAMP, lag=LAG, step=STEP, temp=True)
    dow_train_y = sampler_stamp2(dow_train, stamp=STAMP, lag=LAG, step=STEP, temp=True)
    temporal_train = np.concatenate((dow_train_y, hour_1_train_y), axis=-1)
    
    np.savez(f'./data/x_train_stamp{STAMP}_lag{LAG}_step{STEP}_v2.npz', x_train)
    np.savez(f'./data/y_train_stamp{STAMP}_lag{LAG}_step{STEP}_v2.npz', y_train)
    np.savez(f'./data/temporal_train_stamp{STAMP}_lag{LAG}_step{STEP}_v2.npz', temporal_train)
    
    # test
    x_test, y_test = sampler_stamp2(test_dataset, stamp=STAMP, lag=LAG, step=STEP, temp=False)
    hour_1_test_y = sampler_stamp2(hour_1_test, stamp=STAMP, lag=LAG, step=STEP, temp=True)
    dow_test_y = sampler_stamp2(dow_test, stamp=STAMP, lag=LAG, step=STEP, temp=True)
    temporal_test = np.concatenate((dow_test_y, hour_1_test_y), axis=-1)

    np.savez(f'./data/x_test_stamp{STAMP}_lag{LAG}_step{STEP}_v2.npz', x_test)
    np.savez(f'./data/y_test_stamp{STAMP}_lag{LAG}_step{STEP}_v2.npz', y_test)
    np.savez(f'./data/temporal_test_stamp{STAMP}_lag{LAG}_step{STEP}_v2.npz', temporal_test)

In [9]:
time_unit = [1,2,3,4,6,8,12,24]
stamp_list = time_unit
step_list = [int(24/i) for i in stamp_list]
lag_list = [2*(i+1) for i in range(12)]

for i in range(len(time_unit)): # time unit에 따라
    for j in range(len(lag_list)): # lag에 따라
        save_data_stamp2(STAMP=stamp_list[i], LAG=lag_list[j], STEP=step_list[i])

Sampler Return (2134, 10, 20, 2) (2134, 10, 20, 1)
Sampler Return (0,) (2134, 1, 24)
Sampler Return (0,) (2134, 1, 7)
Sampler Return (694, 10, 20, 2) (694, 10, 20, 1)
Sampler Return (0,) (694, 1, 24)
Sampler Return (0,) (694, 1, 7)
Sampler Return (2132, 10, 20, 4) (2132, 10, 20, 1)
Sampler Return (0,) (2132, 1, 24)
Sampler Return (0,) (2132, 1, 7)
Sampler Return (692, 10, 20, 4) (692, 10, 20, 1)
Sampler Return (0,) (692, 1, 24)
Sampler Return (0,) (692, 1, 7)
Sampler Return (2130, 10, 20, 6) (2130, 10, 20, 1)
Sampler Return (0,) (2130, 1, 24)
Sampler Return (0,) (2130, 1, 7)
Sampler Return (690, 10, 20, 6) (690, 10, 20, 1)
Sampler Return (0,) (690, 1, 24)
Sampler Return (0,) (690, 1, 7)
Sampler Return (2128, 10, 20, 8) (2128, 10, 20, 1)
Sampler Return (0,) (2128, 1, 24)
Sampler Return (0,) (2128, 1, 7)
Sampler Return (688, 10, 20, 8) (688, 10, 20, 1)
Sampler Return (0,) (688, 1, 24)
Sampler Return (0,) (688, 1, 7)
Sampler Return (2126, 10, 20, 10) (2126, 10, 20, 1)
Sampler Return (0,) 

Sampler Return (640, 10, 20, 24) (640, 10, 20, 1)
Sampler Return (0,) (640, 1, 24)
Sampler Return (0,) (640, 1, 7)
Sampler Return (2146, 10, 20, 2) (2146, 10, 20, 1)
Sampler Return (0,) (2146, 1, 24)
Sampler Return (0,) (2146, 1, 7)
Sampler Return (706, 10, 20, 2) (706, 10, 20, 1)
Sampler Return (0,) (706, 1, 24)
Sampler Return (0,) (706, 1, 7)
Sampler Return (2138, 10, 20, 4) (2138, 10, 20, 1)
Sampler Return (0,) (2138, 1, 24)
Sampler Return (0,) (2138, 1, 7)
Sampler Return (698, 10, 20, 4) (698, 10, 20, 1)
Sampler Return (0,) (698, 1, 24)
Sampler Return (0,) (698, 1, 7)
Sampler Return (2130, 10, 20, 6) (2130, 10, 20, 1)
Sampler Return (0,) (2130, 1, 24)
Sampler Return (0,) (2130, 1, 7)
Sampler Return (690, 10, 20, 6) (690, 10, 20, 1)
Sampler Return (0,) (690, 1, 24)
Sampler Return (0,) (690, 1, 7)
Sampler Return (2122, 10, 20, 8) (2122, 10, 20, 1)
Sampler Return (0,) (2122, 1, 24)
Sampler Return (0,) (2122, 1, 7)
Sampler Return (682, 10, 20, 8) (682, 10, 20, 1)
Sampler Return (0,) (6

Sampler Return (1965, 10, 20, 24) (1965, 10, 20, 1)
Sampler Return (0,) (1965, 1, 24)
Sampler Return (0,) (1965, 1, 7)
Sampler Return (525, 10, 20, 24) (525, 10, 20, 1)
Sampler Return (0,) (525, 1, 24)
Sampler Return (0,) (525, 1, 7)
Sampler Return (2134, 10, 20, 2) (2134, 10, 20, 1)
Sampler Return (0,) (2134, 1, 24)
Sampler Return (0,) (2134, 1, 7)
Sampler Return (694, 10, 20, 2) (694, 10, 20, 1)
Sampler Return (0,) (694, 1, 24)
Sampler Return (0,) (694, 1, 7)
Sampler Return (2110, 10, 20, 4) (2110, 10, 20, 1)
Sampler Return (0,) (2110, 1, 24)
Sampler Return (0,) (2110, 1, 7)
Sampler Return (670, 10, 20, 4) (670, 10, 20, 1)
Sampler Return (0,) (670, 1, 24)
Sampler Return (0,) (670, 1, 7)
Sampler Return (2086, 10, 20, 6) (2086, 10, 20, 1)
Sampler Return (0,) (2086, 1, 24)
Sampler Return (0,) (2086, 1, 7)
Sampler Return (646, 10, 20, 6) (646, 10, 20, 1)
Sampler Return (0,) (646, 1, 24)
Sampler Return (0,) (646, 1, 7)
Sampler Return (2062, 10, 20, 8) (2062, 10, 20, 1)
Sampler Return (0,)